# Sentinel-1 burst based processing system at DL

In [1]:
# Work on latest iteration on AWS
import descarteslabs as dl

In [2]:
from descarteslabs.catalog import Product, Image, Search, properties as pr
import matplotlib.pyplot as plt
import numpy as np
import datetime
from shapely.geometry import box

## 1. Overview of Sentinel-1 burst index

1. Product that contains Sentinel-1 IW burst footprints
2. Updated on the fly as the processing system sees new SLCs released by ESA
3. Also, mapped to ESA burst ids

### 1.1 Why maintain own burst footprint database?

1. Independent QA on live stream of SLCs
2. Faster identification of problematic SLC granules
3. DL developed its database before ESA released its database, and our system is more resilient to metadata errors

In [3]:
# Burst footprint product
fp_prod = Product.get("esa:sentinel-1:burst_index:v1")
fp_prod

Product: Sentinel-1 Burst Footprints
  id: esa:sentinel-1:burst_index:v1
  created: Wed Aug 10 02:44:05 2022

In [4]:
# Total number of footprints
fp_prod.images().count()

368479

In [5]:
# IW footprints
fp_prod.images().filter(pr.provider_id > "F").count()   # More than EW

342023

In [6]:
# Query a single burst
Image.get(fp_prod.id + ":" + "071-2637-IW1-VV-RD")

Image: 071-2637-IW1-VV-RD
  id: esa:sentinel-1:burst_index:v1:071-2637-IW1-VV-RD
  product: esa:sentinel-1:burst_index:v1
  created: Wed Aug 10 09:30:16 2022

In [7]:
# Query using ESA-id
for ii in fp_prod.images().filter(pr.provider_id == "IW1-0151226").limit(1):
    print(ii)

Image: 071-2637-IW1-VV-RD
  id: esa:sentinel-1:burst_index:v1:071-2637-IW1-VV-RD
  product: esa:sentinel-1:burst_index:v1
  created: Wed Aug 10 09:30:16 2022


In [8]:
# Extra properties of a footprint
Image.get(fp_prod.id + ":" + "071-2637-IW1-VV-RD").extra_properties

{'anxtime': 2401.00422122095, 'epsg_code': 32611, 'has_land': 1}

## 2. Overview of burst SLC database

1. Product that contains metadata for all IW bursts from Sentinel-1 (V transmit collection is complete)
2. `-SV-` in name indicates single pol data and `-DV-` indicates dual pol data
3. Underlying product that lets DL orchestrate scalable Sentinel-1 backscatter/insar/GSLC pipelines

In [9]:
# Burst SLC product
slc_prod = Product.get("esa:sentinel-1:burst_slc:v1")
slc_prod

Product: Sentinel-1 Burst SLC Index Product
  id: esa:sentinel-1:burst_slc:v1
  created: Tue Mar 14 21:20:34 2023

In [10]:
# Total number of images
slc_prod.images().count()

56356691

In [11]:
# Burst in a given time period - use strings or datetime
slc_prod.images().filter("2023-01-01" < pr.acquired < "2023-01-01T12:00:00").count()

6516

In [12]:
# Burst in a given time period over an AOI - use any shapely polygon
slc_prod.images().intersects(box(70., 15., 75., 20.)).filter("2023-01-01" < pr.acquired < "2023-01-01T12:00:00").count()

90

In [13]:
# Extra properties of a burst
for ii in slc_prod.images().filter("2023-01-01" < pr.acquired < "2023-01-01T12:00:00").limit(1):
    print(ii.id)
    print(ii.extra_properties)

esa:sentinel-1:burst_slc:v1:2023-01-01-S1A-136-2493-IW1-DV-RD-E056
{'absburstid': 'IW1-100055603', 'anx': 2281.451759, 'burstid': 'IW1-290798', 'burstnumber': 1, 'ipf': '003.52', 'line0': 0, 'line1': 1496, 'safe': 'S1A_IW_SLC__1SDV_20230101T004905_20230101T004935_046583_059524_E056.zip', 'tiff': 's1a-iw1-slc-vv-20230101t004905-20230101t004933-046583-059524-004.tiff,s1a-iw1-slc-vh-20230101t004905-20230101t004933-046583-059524-001.tiff'}


## 3. Stacks of bursts

In [14]:
# Query stack by burst-id
slc_prod.images().filter(pr.provider_id == "071-2637-IW1-VV-RD").count()

321

In [15]:
# Query stack by burst-id and time interval
slc_prod.images().filter(pr.provider_id == "071-2637-IW1-VV-RD").filter(pr.acquired > "2022-01-01").count()

73

In [16]:
# Get metadata of images in stack
for ii in slc_prod.images().filter(pr.provider_id == "071-2637-IW1-VV-RD").filter(pr.acquired > "2022-01-01"):
    print(ii.name, ii.extra_properties["ipf"])

2022-01-01-S1A-071-2637-IW1-DV-RD-68E0 003.40
2022-01-13-S1A-071-2637-IW1-DV-RD-18C1 003.40
2022-01-25-S1A-071-2637-IW1-DV-RD-0830 003.40
2022-02-06-S1A-071-2637-IW1-DV-RD-0E4C 003.40
2022-02-18-S1A-071-2637-IW1-DV-RD-81E6 003.40
2022-03-02-S1A-071-2637-IW1-DV-RD-E65B 003.40
2022-03-14-S1A-071-2637-IW1-DV-RD-CC06 003.40
2022-03-26-S1A-071-2637-IW1-DV-RD-6EE3 003.51
2022-04-07-S1A-071-2637-IW1-DV-RD-3469 003.51
2022-04-19-S1A-071-2637-IW1-DV-RD-DD4E 003.51
2022-05-01-S1A-071-2637-IW1-DV-RD-72F9 003.51
2022-05-13-S1A-071-2637-IW1-DV-RD-A1C7 003.52
2022-05-25-S1A-071-2637-IW1-DV-RD-C810 003.52
2022-06-06-S1A-071-2637-IW1-DV-RD-90E6 003.52
2022-06-18-S1A-071-2637-IW1-DV-RD-B558 003.52
2022-06-30-S1A-071-2637-IW1-DV-RD-FD13 003.52
2022-07-12-S1A-071-2637-IW1-DV-RD-69C7 003.52
2022-07-24-S1A-071-2637-IW1-DV-RD-996C 003.52
2022-08-05-S1A-071-2637-IW1-DV-RD-7B86 003.52
2022-08-17-S1A-071-2637-IW1-DV-RD-ECF9 003.52
2022-08-29-S1A-071-2637-IW1-DV-RD-6C6B 003.52
2022-09-10-S1A-071-2637-IW1-DV-RD-

## 4. API for accessing burst metadata

1. Ask to get access to the internal `s1geocoder` repo
2. Install the repo and set up other necessary AWS credentials

Typical call to get metadata will look like this:

```python
from s1geocoder.s1parser.dl import bursts

meta_zip, orbit_file = bursts.stage_metadata(slc_img, **creds)
burst, lookup = bursts.load_burst(meta_zip, orbit_file=orbit_file, pol=pol)
```

## 5. API for accessing burst imagery

Once, you have access to the metadata of a burst, you can also extract the imagery of a single burst to a numpy array as follows:

```python
from s1geocoder.s1parser.dl import bursts

stream = bursts.get_burst_imagery(slc_img, burst, lookup, **creds)
img0 = (stream[::2] + 1j * stream[1::2]).reshape((burst.numberOfLines, burst.numberOfSamples)).astype(np.complex64)
```

## 6. Submitting GSLC jobs to the GDS system

1. Request access to GDS enpoint and provide AWS role information
2. Submit jobs to geocode via API
3. Check catalog product after a while and geocoded bursts should be available like any other data there

Here is a sample code snippet that lets you submit jobs to GDS.

```python
import requests

gc_url = os.environ.get("GDS_GEOCODER_ENDPOINT")

burst_footprint = "045-3867-IW1-VV-RD"

resp2 = requests.post(
    f"{gc_url}/gds/geocode/footprint/",
    params={
        "footprint": burst_footprint,
        "start": "2018-01-01",
        "stop": "2019-01-01",
    },
    json=["testrun", "example"],   # Tag this with something meaningful
    auth=get_creds(),
)

print(resp2.json())
```